In [1]:

from keras import backend as K
from keras.layers import Layer
from keras import initializers, regularizers, constraints
 
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
 
 
class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """
 
    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
 
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')
 
        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
 
        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)
 
        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)
 
    def build(self, input_shape):
        assert len(input_shape) == 3
 
        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
 
        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)
 
        super(AttentionWithContext, self).build(input_shape)
 
    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None
 
    def call(self, x, mask=None):
        uit = dot_product(x, self.W)
 
        if self.bias:
            uit += self.b
 
        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)
 
        a = K.exp(ait)
 
        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())
 
        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
 
        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)
 
    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import time


from keras.callbacks import  ModelCheckpoint
from keras.callbacks import  CSVLogger
from keras.callbacks import  EarlyStopping

import pandas as pd
import os
from judge_callback import  *
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

data=pd.read_csv('./csv/Big_gen.csv')
data.head()

,Unnamed: 0,accusation,death_penalty,fact,imprisonment,life_imprisonment,relevant_articles_label,fact_cut_wd,imp_label
0,0,['盗窃'],False,公诉机关指控：2016年3月28日20时许，被告人颜某在本市洪山区马湖新村足球场马路边捡拾到...,4,False,['264'],公诉 机关 指控 被告人 某某 马湖 足球场 马路边 捡拾 被害人 某某 CHEPAIHAO...,6
1,1,['盗窃'],False,天津市静海县人民检察院指控，2014年5月13日上午8时许，被告人李xx在天津市静海县大邱庄...,0,False,['264'],人民检察院 指控 上午 被告人 某某 大邱庄 宾馆 227 房间 某某 配货站 趁该 配货站...,2
2,2,['强奸'],False,永顺县人民检察院指控，2014年1月11日，被告人李某某与彭某某（另案处理）在永顺县塔卧镇“...,144,False,['236'],人民检察院 指控 被告人 某某 某某 另案处理 宾馆 房间内 先后 强行 被害人 某某 发生...,21
3,3,['盗窃'],False,公诉机关起诉书指控：\r\n2016年11月17日凌晨1时许，被告人周某在本县武康街道营盘小...,6,False,['264'],公诉 机关 起诉书 指控 凌晨 被告人 某某 武康 街道 营盘 131 南侧 过道 采用 拉...,7
4,4,['故意伤害'],False,大名县人民检察院起诉书指控，2014年3月25日9时许，被告人张某在自家庄某处因故与本村席某...,21,False,['234'],人民检察院 起诉书 指控 被告人 某某 自家 某某 因故 某某 发生争执 引起 打架 打架 ...,12


In [ ]:
from keras.layers import  Input,Embedding,Bidirectional,LSTM,GlobalMaxPool1D,GlobalAveragePooling1D,concatenate,Dense,Activation
from keras.models import  Model
from keras.layers import  Dropout,CuDNNLSTM,CuDNNGRU,BatchNormalization,Conv1D,MaxPool1D,Flatten
max_features=200000
maxlen=1000
embed_size=300




def pro_text_cnn_model():
    inp = Input(shape=(maxlen,))
    embed= Embedding(max_features, embed_size)(inp)

    # cnn1模块，kernel_size = 3
    conv1_1 = Conv1D(256, 3, padding='same')(embed)
    bn1_1 = BatchNormalization()(conv1_1)
    relu1_1 = Activation('relu')(bn1_1)
    conv1_2 = Conv1D(128, 3, padding='same')(relu1_1)
    bn1_2 = BatchNormalization()(conv1_2)
    relu1_2 = Activation('relu')(bn1_2)
    cnn1 = MaxPool1D(pool_size=4)(relu1_2)
    # cnn2模块，kernel_size = 4
    conv2_1 = Conv1D(256, 4, padding='same')(embed)
    bn2_1 = BatchNormalization()(conv2_1)
    relu2_1 = Activation('relu')(bn2_1)
    conv2_2 = Conv1D(128, 4, padding='same')(relu2_1)
    bn2_2 = BatchNormalization()(conv2_2)
    relu2_2 = Activation('relu')(bn2_2)
    cnn2 = MaxPool1D(pool_size=4)(relu2_2)
    # cnn3模块，kernel_size = 5
    conv3_1 = Conv1D(256, 5, padding='same')(embed)
    bn3_1 = BatchNormalization()(conv3_1)
    relu3_1 = Activation('relu')(bn3_1)
    conv3_2 = Conv1D(128, 5, padding='same')(relu3_1)
    bn3_2 = BatchNormalization()(conv3_2)
    relu3_2 = Activation('relu')(bn3_2)
    cnn3 = MaxPool1D(pool_size=4)(relu3_2)
    # 拼接三个模块
    cnn = concatenate([cnn1,cnn2,cnn3], axis=-1)
    flat = Flatten()(cnn)
    drop = Dropout(0.5)(flat)
    fc = Dense(512)(drop)
    bn = BatchNormalization()(fc)
    main_output = Dense(25, activation='softmax')(bn)
    model = Model(inputs = inp, outputs = main_output)
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

In [5]:
model=pro_text_cnn_model()

In [7]:

from sklearn.utils import  shuffle
from keras.preprocessing.text import  Tokenizer
from keras.preprocessing.sequence import  pad_sequences



import os 
import pickle
if os.path.exists("./model/Tokenizer.pkl"):
    with open('./model/Tokenizer.pkl','rb') as f:
        tokenizer=pickle.load(f)
else:
    tokenizer=Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(data['fact_cut_wd'].values)
    with open("./model/Tokenizer.pkl",'wb') as f:
        pickle.dump(f,tokenizer)
    
data=shuffle(data)

train_data=data

train_data_text_list=train_data['fact_cut_wd'].values
train_data_seq=tokenizer.texts_to_sequences(train_data_text_list)
train_x=pad_sequences(train_data_seq,maxlen)

In [8]:
from keras.utils import  to_categorical
from keras.callbacks import   EarlyStopping,CSVLogger


train_y=train_data['imp_label'].values
train_y=to_categorical(train_y.reshape(-1,1)) 

In [9]:
 
    
model.fit(train_x,train_y,class_weight='auto',epochs=6,batch_size=256,validation_split=0.02,callbacks=[EarlyStopping(),
                                CSVLogger(filename="./log/result.csv")])   

Train on 2064123 samples, validate on 42125 samples
Epoch 1/6
2064123/2064123 [==============================] - 4170s 2ms/step - loss: 1.9819 - acc: 0.3224 - val_loss: 3.2925 - val_acc: 0.1189
Epoch 2/6
 232960/2064123 [==>...........................] - ETA: 1:01:14 - loss: 1.7946 - acc: 0.3697

KeyboardInterrupt: 

In [9]:

model.save("./model/text_cnn_time.model")